# LightGBMの推論用カスタムコンテナを構築し、SageMakerによる推論の仕組みを深く理解する

## アジェンダ
* パターン3のカスタムコンテナを作成
    * 学習編で実施済み
* パターン2のカスタムコンテナを作成：コンテナファイルを外部から指定する
    * Inference-Toolkitと、MMSをインストールしたコンテナを作成
* XGBoostビルトインコンテナで、LightBGMの推論を実施する
    * requirements.txtを用意する
    * inference.pyをお作用に従って作る

# Lab : LightGBMのカスタムコンテナを通して、SageMakerの動作を理解する

LightGBMがインストールされたカスタムコンテナを構築し、SageMaker Trainingジョブで学習を行います。
カスタムコンテナの挙動を観察し、SageMakerの動作について理解を深めます。

ノートブックは20分程度で実行できます。

# 0.実行環境確認
本ノートブックは、SageMakerノートブックインスタンス上で動作確認しています。
* インスタンスタイプ：ml.t3.medium
* カーネル：conda_python3

## 0-1.pythonバージョン確認

In [88]:
#Pythonのバージョン情報
import sys
sys.version # 3.8.12

'3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:59:51) \n[GCC 9.4.0]'

In [87]:
# Pythonのバージョン確認 (システムコマンド使用）
!python -V # 3.8.12

Python 3.8.12


## 0-2.SageMakerSDKバージョン確認

Amazon SageMaker Python SDKは、Amazon SageMaker上で機械学習されたモデルをトレーニングおよびデプロイするためのオープンソースライブラリです。

このSDKを使用すると、一般的な深層学習フレームワーク、Amazonが提供するアルゴリズム、またはSageMaker互換のDockerイメージに組み込まれた独自のアルゴリズムを使ってモデルをトレーニングおよびデプロイすることができます。

* ドキュメント : https://sagemaker.readthedocs.io/en/stable/
* GitHub : https://github.com/aws/sagemaker-python-sdk

SageMakerSDK をインポートすると、バケットが作成されます。  
sagemaker-＜region＞-＜account＞

In [3]:
# SageMakerSDK のバージョン確認
import sagemaker
print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__)) # 2.110.0

Current SageMaker Python SDK Version =2.101.1


# 1.データ準備

学習、推論で利用するデータを準備します。

scikit-learn付属の、ボストン住宅価格データセットを利用します。(注：バージョン1.2から除外されます）  
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

以下のスクリプトを参考にしています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/lightgbm_bring_your_own_container_local_training_and_serving/lightgbm_bring_your_own_container_local_training_and_serving.py

In [4]:
import sklearn
sklearn.__version__ # 1.0.1

'1.0.1'

In [5]:
import pandas as pd
pd.__version__ # 1.3.4

'1.3.4'

## 1-1. データロード

In [6]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [7]:
data = load_boston() # 1.2でデータセットがなくすという警告が出ますが動作に影響ありません

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

## 1-2. 特徴量生成（Feature Engineering）
本ノートブックでは実施しません。そのままデータを利用します。

## 1-3. データ分割
学習用（train）、評価用（validation）、テスト用（test）にデータを分割します。  
train:val:test = 3(60%):1(20%):1(20%)に分割します。  

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=45)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=45)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

valX = pd.DataFrame(X_val, columns=data.feature_names)
valX['target'] = y_val

testX = pd.DataFrame(X_test, columns=data.feature_names)

In [9]:
# 確認
trainX.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
1,0.33983,22.0,5.86,0.0,0.431,6.108,34.9,8.0555,7.0,330.0,19.1,390.18,9.16,24.3
2,0.10469,40.0,6.41,1.0,0.447,7.267,49.0,4.7872,4.0,254.0,17.6,389.25,6.05,33.2
3,6.80117,0.0,18.10,0.0,0.713,6.081,84.4,2.7175,24.0,666.0,20.2,396.90,14.70,20.0
4,1.35472,0.0,8.14,0.0,0.538,6.072,100.0,4.1750,4.0,307.0,21.0,376.73,13.04,14.5


In [10]:
# 確認
valX.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.03150,95.0,1.47,0.0,0.403,6.975,15.3,7.6534,3.0,402.0,17.0,396.90,4.56,34.9
1,0.51183,0.0,6.20,0.0,0.507,7.358,71.6,4.1480,8.0,307.0,17.4,390.07,4.73,31.5
2,19.60910,0.0,18.10,0.0,0.671,7.313,97.9,1.3163,24.0,666.0,20.2,396.90,13.44,15.0
3,0.95577,0.0,8.14,0.0,0.538,6.047,88.8,4.4534,4.0,307.0,21.0,306.38,17.28,14.8
4,0.09604,40.0,6.41,0.0,0.447,6.854,42.8,4.2673,4.0,254.0,17.6,396.90,2.98,32.0


In [11]:
# 確認
testX.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.25387,0.0,6.91,0.0,0.4480,5.399,95.3,5.8700,3.0,233.0,17.9,396.90,30.81
1,0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
2,4.64689,0.0,18.10,0.0,0.6140,6.980,67.6,2.5329,24.0,666.0,20.2,374.68,11.66
3,3.67367,0.0,18.10,0.0,0.5830,6.312,51.9,3.9917,24.0,666.0,20.2,388.62,10.58
4,0.29819,0.0,6.20,0.0,0.5040,7.686,17.0,3.3751,8.0,307.0,17.4,377.51,3.92


In [12]:
# 確認
y_test[0:5]

array([14.4, 33. , 29.8, 21.2, 46.7])

## 1-4.データ保存
ローカル、S3それぞれにデータを保存します。

### 1-4-1.ローカルへ保存

In [13]:
# ディレクトリ作成
from pathlib import Path

Path('./data/train').mkdir(parents=True, exist_ok=True)
Path('./data/valid').mkdir(parents=True, exist_ok=True)
Path('./data/test').mkdir(parents=True, exist_ok=True)

In [14]:
# ローカルへ保存
local_train = './data/train/boston_train.csv'
local_valid = './data/valid/boston_valid.csv'
local_test = './data/test/boston_test.csv'

trainX.to_csv(local_train, header=None, index=False)
valX.to_csv(local_valid, header=None, index=False)
testX.to_csv(local_test, header=None, index=False)

### 1-4-2.S3へ保存

一意のバケット作成のために、sgemaker.Session().default_bucket()を利用します。

https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session

sagemaker-＜region＞-＜accoutid＞　を取得することができます。

In [15]:
bucket_name = sagemaker.Session().default_bucket()
region_name = sagemaker.Session().boto_region_name
account_id =  sagemaker.Session().account_id()

In [16]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

sagemaker-ap-northeast-1-805433377179
ap-northeast-1
805433377179


In [17]:
# バケット作成(SageMakerSDKのインポート時作成されています。他のバケット作成時に利用ください)
#import boto3

#s3_resource = boto3.resource('s3')
#s3_resource.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region_name})

In [18]:
# S3へ保存
train_s3 = sagemaker.s3.S3Uploader.upload('./data/train/boston_train.csv', f's3://{bucket_name}/demo_lightgbm/train')
valid_s3 = sagemaker.s3.S3Uploader.upload('./data/valid/boston_valid.csv', f's3://{bucket_name}/demo_lightgbm/valid')

# 2.LightGBMカスタムコンテナの構築


カスタムコンテナの作成には大きく分けて3つのパターンがあります。詳細は以下のブログを参考ください。

https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-training/

SageMakerの動作を理解するためにパターン3のベースイメージ + カスタムレイヤー方式を採用します。

## 2-1. Dockerfileの確認

資材はこちらのノートブックを参考に準備しています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/tree/main/lightgbm_bring_your_own_container_local_training_and_serving/container

In [ ]:
!pygmentize ./container/Dockerfile

## 2-2. dockerイメージの build & push

ビルド&pushには7分ほどかかります。

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression

cd container

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 2-3. 学習前設定
ECRでpushしたコンテナのURIを確認

AWSコンソールでECRに移動し、作成したコンテナがあることを確認します。

image URIを取得し、以下にはりつけます。

In [19]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

sagemaker-ap-northeast-1-805433377179
ap-northeast-1
805433377179


In [20]:
image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sagemaker-lightgbm-regression'

In [21]:
# 確認
image_uri

'805433377179.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-lightgbm-regression'

In [22]:
hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0}

## 2-4.ローカル学習の実行
ECRからビルドしたイメージを持ってきて、ローカルのdockerでビルドして、実行する

In [ ]:
# ローカルファイルのパスを設定（S3パス指定も可）
train_location = 'file://'+local_train
valid_location = 'file://'+local_valid

print(train_location)
print(valid_location)

In [27]:
from sagemaker.estimator import Estimator

In [28]:
from sagemaker import get_execution_role

role = get_execution_role()

In [29]:
# 確認
role

'arn:aws:iam::805433377179:role/service-role/AmazonSageMaker-ExecutionRole-20220807T102095'

SageMakerのEstimatorを作成します。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html

In [30]:
local_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters
    )

fitメソッドで学習ジョブを発行します

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

In [ ]:
local_lightgbm.fit({'train':train_location, 'validation': valid_location})

ローカルモードの学習結果は

Amazon S3
Buckets
sagemaker-us-west-2-805433377179
sagemaker-lightgbm-regression-2022-10-03-06-17-32-054/

に出力されます。


## 2-5.ローカルデプロイ

serializer : インプットデータの形式を指定します。
https://sagemaker.readthedocs.io/en/stable/v2.html

In [ ]:
local_predictor = local_lightgbm.deploy(1, 'local', serializer=sagemaker.serializers.CSVSerializer()) 

In [ ]:
!docker ps

In [ ]:
# 起動中のコンテナを停止する場合
#!docker stop XXXXXXXXXXX #XXXXXXXXXXXは CONtAINER ID
#!docker ps

## 2-6.ローカルエンドポイントで推論実施

In [ ]:
# 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = local_predictor.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

## 2-7.学習ジョブを発行
次は、ローカルモードではなく、
同じカスタムコンテナで、学習ジョブを実行します。

In [ ]:
# 確認
print(train_s3)
print(valid_s3)

In [ ]:
est_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge", # インスタンスタイプを指定
    hyperparameters=hyperparameters)

In [ ]:
est_lightgbm.fit({'train':train_s3, 'validation': valid_s3})

学習には3分ほど時間がかかります。

課金されるのは75秒ほどです。

## 2-8.エンドポイントにデプロイ

デプロイすると、
SageMaker は docker run <image> serveを実行します。

    
デプロイには3分ほどかかります。

In [23]:
from sagemaker.predictor import csv_serializer

deployメソッドで、推論エンドポイントをデプロイします。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy

In [ ]:
predictor = est_lightgbm.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=True)

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor.predict(payload).decode('utf-8')
print(predicted)

# 3. 実行ファイルを外部から指定する

「2.LightGBMカスタムコンテナの構築」ではカスタムコンテナ内に学習起動スクリプトtrainを配置しましたが、
ソースコードを修正するごとにコンテナを作り替える必要があります。

保守性を上げるには、コンテナ（環境）とソースコードを分けた方がいい場合もあります。
以下では外部からスクリプトファイルを指定する方法を紹介します。

## 3-0.SageMaker Training Toolkitとは
外部からスクリプトを指定するためには、SageMaker Training Toolkitを導入します。

https://github.com/aws/sagemaker-training-toolkit


trainコマンドが  
/opt/conca/bin/train  
にインストールされます。  


先程のdockerfileに追記します。
資材からは、trainを除外しておきます。trainを含んだままの場合、
docker run <image> train
を実行したときに、カレントディレクトリのtrainスクリプトが実行されてしまい、training toolkitが導入した　trainコマンドが実行できないためです。

## 3-1. Dockerfile確認

In [ ]:
!pygmentize ./container_smtrtoolkit/Dockerfile

## 3-2. build & push

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-toolkit

#cd container
cd container_smtrtoolkit ### 変更点

#chmod +x lightgbm_regression/train ### 変更点
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


## 3-3.学習(ローカル)

In [ ]:
image_uri_toolkit = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sagemaker-toolkit'

In [ ]:
# 確認
image_uri_toolkit

In [ ]:
est_lightgbm_toolkit = Estimator(
    image_uri_toolkit,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
    entry_point='./src/train_practice.py'
    )

In [ ]:
est_lightgbm_toolkit.fit({'train':train_s3, 'validation': valid_s3})

ローカルモードで学習することができました。別のスクリプトを指定してみましょう。

In [ ]:
est_lightgbm_toolkit2 = Estimator(
    image_uri_toolkit,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
    #entry_point='./src/train_practice.py'
    entry_point='./src/train_practice.sh' ### シェルスクリプトに変更
    )

In [ ]:
est_lightgbm_toolkit2.fit({'train':train_s3, 'validation': valid_s3})

コンテナ外部から任意のファイルを実行することが確認できました。

## （optional）4. カスタムコンテナを使わず、built-inコンテナのrequirement.txtにlightgbmを記載して実行する



過去バージョン（1.3-3, 1.2-2, 1.2-1, 1.0-1)はこちら

https://github.com/aws/sagemaker-xgboost-container/releases


In [95]:
container_uri = sagemaker.image_uris.retrieve("xgboost", region_name, "1.5-1")

In [96]:
container_uri

'354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1'

In [89]:
est_xgb = Estimator(
    container_uri, # xgboostのbuilt-inコンテナ
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
    source_dir='./src_builtin_container',
    entry_point='train_practice.py'
    )

In [90]:
[1,2,3,4,5]

[1, 2, 3, 4, 5]

In [91]:
est_xgb.fit({'train':train_s3, 'validation': valid_s3})

Creating sxjw7uohko-algo-1-h7sa7 ... 
Creating sxjw7uohko-algo-1-h7sa7 ... done
Attaching to sxjw7uohko-algo-1-h7sa7
sxjw7uohko-algo-1-h7sa7 | [2022-10-17 00:34:21.420 d95ccc8b7471:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
sxjw7uohko-algo-1-h7sa7 | [2022-10-17:00:34:21:INFO] Imported framework sagemaker_xgboost_container.training
sxjw7uohko-algo-1-h7sa7 | [2022-10-17:00:34:21:INFO] Failed to parse hyperparameter boosting_type value gbdt to Json.
sxjw7uohko-algo-1-h7sa7 | Returning the value itself
sxjw7uohko-algo-1-h7sa7 | [2022-10-17:00:34:21:INFO] Failed to parse hyperparameter objective value regression to Json.
sxjw7uohko-algo-1-h7sa7 | Returning the value itself
sxjw7uohko-algo-1-h7sa7 | [2022-10-17:00:34:21:INFO] No GPUs detected (normal if no gpus installed)
sxjw7uohko-algo-1-h7sa7 | [2022-10-17:00:34:21:INFO] Invoking user training script.
sxjw7uohko-algo-1-h7sa7 | [2022-10-17:00:34:21:INFO] Module train_practice does not provide a setup.py. 
sxjw7uohko-algo-1-h7s

## 4-2. 推論実施

### 4-2-1.デプロイ

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy


デプロイの際に、ソースコードを指定するにはどうしたらいいのか？

https://www.youtube.com/watch?v=sngNd79GpmE&t=596s


ポイント：あらためて、Estimatorを定義する必要がある。

In [92]:
# 確認
print(est_xgb)
print(est_xgb.base_job_name)
print(est_xgb.checkpoint_local_path)
print(est_xgb.checkpoint_s3_uri)
print(est_xgb.code_channel_name)
print(est_xgb.code_location)
print(est_xgb.code_uri)
print(est_xgb.collection_configs)
print(est_xgb.CONTAINER_CODE_CHANNEL_SOURCEDIR_PATH)
print(est_xgb.container_log_level)
print(est_xgb.debugger_hook_config)
print(est_xgb.debugger_rule_configs)
print(est_xgb.debugger_rules)
print(est_xgb.dependencies)
print(est_xgb.deploy_instance_type)
print(est_xgb.disable_profiler)
print(est_xgb.enable_sagemaker_metrics)
print(est_xgb.encrypt_inter_container_traffic)
print(est_xgb.entry_point)
print(est_xgb.environment)
print(est_xgb.git_config)
print(est_xgb.image_uri)
print(est_xgb.input_mode)
print(est_xgb.instance_count)
print(est_xgb.model_uri)
print(est_xgb.model_data)


sagemaker-xgboost
None
None
code
None
None
set()
/opt/ml/input/data/code/sourcedir.tar.gz
20
None
[]
[]
[]
None
False
None
False
train_practice.py
None
None
354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1
File
1
None
s3://sagemaker-ap-northeast-1-805433377179/sagemaker-xgboost-2022-10-17-00-34-15-146/model.tar.gz


### serve用のファイルは、.py かつ、作法に従う必要がある。

MMSは.pyを扱うように設計されているため。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator

In [93]:
# デプロイ
predictor_xgb = est_xgb.deploy(1, 'local', serializer=csv_serializer, wait=True)

Attaching to 03dbsqo0lj-algo-1-9gst2
03dbsqo0lj-algo-1-9gst2 | [2022-10-17:00:34:40:INFO] No GPUs detected (normal if no gpus installed)
03dbsqo0lj-algo-1-9gst2 | [2022-10-17:00:34:40:INFO] No GPUs detected (normal if no gpus installed)
03dbsqo0lj-algo-1-9gst2 | [2022-10-17:00:34:40:INFO] nginx config: 
03dbsqo0lj-algo-1-9gst2 | worker_processes auto;
03dbsqo0lj-algo-1-9gst2 | daemon off;
03dbsqo0lj-algo-1-9gst2 | pid /tmp/nginx.pid;
03dbsqo0lj-algo-1-9gst2 | error_log  /dev/stderr;
03dbsqo0lj-algo-1-9gst2 | 
03dbsqo0lj-algo-1-9gst2 | worker_rlimit_nofile 4096;
03dbsqo0lj-algo-1-9gst2 | 
03dbsqo0lj-algo-1-9gst2 | events {
03dbsqo0lj-algo-1-9gst2 |   worker_connections 2048;
03dbsqo0lj-algo-1-9gst2 | }
03dbsqo0lj-algo-1-9gst2 | 
03dbsqo0lj-algo-1-9gst2 | http {
03dbsqo0lj-algo-1-9gst2 |   include /etc/nginx/mime.types;
03dbsqo0lj-algo-1-9gst2 |   default_type application/octet-stream;
03dbsqo0lj-algo-1-9gst2 |   access_log /dev/stdout combined;
03dbsqo0lj-algo-1-9gst2 | 
03dbsqo0lj-algo

RuntimeError: Giving up, endpoint didn't launch correctly

In [94]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


## エラー

RuntimeError: Model /opt/ml/model/model.tar.gz cannot be loaded:


6o0805unb3-algo-1-k8ugv | [2022-10-16 02:25:50 +0000] [19] [ERROR] Exception in worker process  
6o0805unb3-algo-1-k8ugv | Traceback (most recent call last):  
6o0805unb3-algo-1-k8ugv |   File "/miniconda3/lib/python3.8/site-packages/sagemaker_xgboost_container/algorithm_mode/serve_utils.py", line 175, in get_loaded_booster  
6o0805unb3-algo-1-k8ugv |     booster = pkl.load(open(full_model_path, "rb"))  
6o0805unb3-algo-1-k8ugv | _pickle.UnpicklingError: invalid load key, '\x1f'.  

In [83]:
# モデルをDLして確認する
print(est_xgb.model_data)

s3://sagemaker-ap-northeast-1-805433377179/sagemaker-xgboost-2022-10-16-02-25-23-725/model.tar.gz


In [84]:
!aws s3 cp $est_xgb.model_data .

download: s3://sagemaker-ap-northeast-1-805433377179/sagemaker-xgboost-2022-10-16-02-25-23-725/model.tar.gz to ./model.tar.gz


In [85]:
!tar zxvf model.tar.gz

lightgbm-regression-model.txt


## 原因
lightgbm-regression-model.txtなので、pklでは読み込めない。

モデルロードする関数を上書きするには？？（そもそもこれがやりたい）

https://github.com/aws/sagemaker-xgboost-container/blob/master/docker/1.5-1/final/Dockerfile.cpu

# Set SageMaker entrypoints
ENV SAGEMAKER_TRAINING_MODULE sagemaker_xgboost_container.training:main  
ENV SAGEMAKER_SERVING_MODULE sagemaker_xgboost_container.serving:main  


まず、serving.main()が実行される

https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L143

serving_env = env.ServingEnv()

で、環境変数にパラメータが読み込まれる


L147

user_module = modules.import_module(serving_env.module_dir, serving_env.module_name)

ここで、ユーザーのモジュールが読み込まれる。

L18をみると、sagemaker_containers.beta.framework.modulesがモジュールのようだ。

from sagemaker_containers.beta.framework import (
    encoders,
    env,
    modules,
    server,
    transformer,
    worker,
)

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/beta/framework/__init__.py

sagemaker_containers.beta.frameworkはアーカイブされている。

現在はこちら。initをみると

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_modules.py



L258で、imortしている。

module = importlib.import_module(name)

def import_module(uri, name=DEFAULT_MODULE_NAME, cache=None):  # type: (str, str, bool) -> module

とあるように、DEFAULT_MODULE_NAMEが読み込まれるようだ





https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L148,149: L147で読み込んだユーザーモジュールに上書きする

user_module_transformer = _user_module_transformer(user_module)  
user_module_transformer.initialize()  


L116にあるように、model_fnなどのユーザー関数に上書きされる。


def _user_module_transformer(user_module):  
    model_fn = getattr(user_module, "model_fn", default_model_fn)  
    input_fn = getattr(user_module, "input_fn", None)  
    predict_fn = getattr(user_module, "predict_fn", None)  
    output_fn = getattr(user_module, "output_fn", None)  
    transform_fn = getattr(user_module, "transform_fn", None)  

## model_fnを定義したファイルが、importされているか？

いま、そもそも環境変数に正しく情報渡せていない気がする。


https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_modules.py

L237より、

def import_module(uri, name=DEFAULT_MODULE_NAME, cache=None):  # type: (str, str, bool) -> module

第二引数に指定する必要がある。

これを呼ぶのは、


https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L147

user_module = modules.import_module(serving_env.module_dir, serving_env.module_name)

serving_env.module_name である。指定できているのか？


L143より

serving_env = env.ServingEnv()

これは、以下のファイル。

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_env.py

L862

class ServingEnv(_Env):



https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_env.py

L329には、

class _Env(_mapping.MappingMixin):


module_name = os.environ.get(_params.USER_PROGRAM_ENV, None)

とある。


L595

TrainingEnvには、

        # override base class attributes  
        if self._module_name is None:  
            self._module_name = str(sagemaker_hyperparameters.get(_params.USER_PROGRAM_PARAM, None))  
        self._user_entry_point = self._user_entry_point or sagemaker_hyperparameters.get(  
            _params.USER_PROGRAM_PARAM  
        )  
        
        
        

## USER_PROGRAM_ENVに設定できればいい？


https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/parameters.py


L18

USER_PROGRAM_ENV = "SAGEMAKER_PROGRAM"  # type: str

SAGEMAKER_PROGRAMに設定できればいいようだ。

ビルトインコンテナにはどうすれば設定できるのだろうか？？

以下のYouTubeだと、boto3でEnvironment引数を使っている。

https://youtu.be/sngNd79GpmE?t=780

# デバッグのために、dockerイメージをプルして、中をみてみる。

ビルトインコンテナの中身をみるには、どうすればいいのか？

XGBoostの場合は、ローカルでbuildしていくようだ。

https://github.com/aws/sagemaker-xgboost-container

In [113]:
container_uri

'354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1'

In [115]:
pwd

'/home/ec2-user/SageMaker/sagemaker_custom_container/byoc_lgbm_serve'

In [ ]:
cd 

In [116]:
# Example

# CPU
!docker build -t xgboost-container-base:1.5-1-cpu-py3 -f /home/ec2-user/SageMaker/sagemaker-xgboost-container/docker/1.5-1/base/Dockerfile.cpu .

Sending build context to Docker daemon  402.5kB
Step 1/23 : ARG UBUNTU_VERSION=18.04
Step 2/23 : ARG CUDA_VERSION=10.2
Step 3/23 : ARG IMAGE_DIGEST=ed4b2e8703ed3f94029ce0823c44fe57b2800108953cf81f97781d6a2a5535fc
Step 4/23 : FROM nvidia/cuda:${CUDA_VERSION}-base-ubuntu${UBUNTU_VERSION}@sha256:${IMAGE_DIGEST}
docker.io/nvidia/cuda:10.2-base-ubuntu18.04@sha256:ed4b2e8703ed3f94029ce0823c44fe57b2800108953cf81f97781d6a2a5535fc: Pulling from nvidia/cuda

8a513d66: Already exists 
96f27655: Already exists 
5003ff51: Already exists 
378426ea: Already exists 
ffd87727: Already exists Digest: sha256:ed4b2e8703ed3f94029ce0823c44fe57b2800108953cf81f97781d6a2a5535fc
Status: Downloaded newer image for nvidia/cuda:10.2-base-ubuntu18.04@sha256:ed4b2e8703ed3f94029ce0823c44fe57b2800108953cf81f97781d6a2a5535fc
 ---> 3cc6331c2c3c
Step 5/23 : ARG MINICONDA_VERSION=4.9.2
 ---> Running in 26dfefbc764c
Removing intermediate container 26dfefbc764c
 ---> 335c5107cda5
Step 6/23 : ARG CONDA_PY_VERSION=39
 ---> Ru

In [118]:
# Example

# CPU
!docker build -t preprod-xgboost-container:1.5-1-cpu-py3 -f /home/ec2-user/SageMaker/sagemaker-xgboost-container/docker/1.5-1/final/Dockerfile.cpu .

Sending build context to Docker daemon  518.8kB
Step 1/30 : ARG SAGEMAKER_XGBOOST_VERSION=1.5-1
Step 2/30 : ARG PYTHON_VERSION=3.8
Step 3/30 : FROM xgboost-container-base:${SAGEMAKER_XGBOOST_VERSION}-cpu-py3
pull access denied for xgboost-container-base, repository does not exist or may require 'docker login': denied: requested access to the resource is denied


# コンテナの中に入って確認する方法
コンソールを立ち上げて、以下の流れで実行する


ディレクトリ移動
 $ cd sagemaker-xgboost-container/
 
baseコンテナをビルド
 $ docker build -t xgboost-container-base:1.5-1-cpu-py3 -f docker/1.5-1/base/Dockerfile.cpu .

finalコンテナをビルド
 $ docker build -t preprod-xgboost-container:1.5-1-cpu-py3 -f docker/1.5-1/final/Dockerfile.cpu .

構築されたイメージを確認
$ docker image ls

中に入って確認（コンテナのタグもつけて指定すること）
$ docker run -it preprod-xgboost-container:1.5-1-cpu-py3 /bin/bash

$ docker run -it 354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1 /bin/bash  

   

# コンテナの中に入り、 command serveを実行してみる

In [99]:
aws ecr get-login-password --region ${your_region} | docker login --username AWS --password-stdin ${your_aws_account_id}.dkr.ecr.${your_region}.amazonaws.com

SyntaxError: invalid syntax (3661551353.py, line 1)

In [98]:
!docker pull $container_uri

Error response from daemon: pull access denied for 354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost, repository does not exist or may require 'docker login': denied: Your authorization token has expired. Reauthenticate and try again.


In [100]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-xgb-builtin

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}



Login Succeeded


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [101]:
!docker pull $container_uri

Error response from daemon: pull access denied for 354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost, repository does not exist or may require 'docker login': denied: Your authorization token has expired. Reauthenticate and try again.


In [102]:
container_uri

'354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1'

In [103]:
!docker pull 354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1

Error response from daemon: pull access denied for 354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost, repository does not exist or may require 'docker login': denied: Your authorization token has expired. Reauthenticate and try again.


In [107]:
!docker login https://354813040037.dkr.ecr.ap-northeast-1.amazonaws.com

Authenticating with existing credentials...
Login did not succeed, error: Error response from daemon: login attempt to https://354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/v2/ failed with status: 403 Forbidden
Username (AWS): ^C


In [59]:
# 改めて、Estimatorを定義する
est_xgb_serve = Estimator(
    container_uri, # xgboostのbuilt-inコンテナ
    role,
    model_data=est_xgb.model_data, ### 学習で保存されたモデルのURI
    instance_count=1,
    instance_type="local",
    #hyperparameters=hyperparameters,
    source_dir='./src_builtin_container_serve',
    entry_point='serve.py' 
    )

In [60]:
# デプロイ
predictor_xgb = est_xgb_serve.deploy(1, 'local', serializer=csv_serializer, wait=True)

ValueError: Estimator is not associated with a training job

ValueError: Estimator is not associated with a training job

# エラー　trainingjobとの紐付け

https://stackoverflow.com/questions/63340328/how-to-define-a-sagemaker-estimator-object-using-a-pre-trained-model-and-then-de

In [ ]:
est_xgb_serve2 = est_xgb_serve.attach(TrainingJobName)

In [57]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [58]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_xgb.predict(payload).decode('utf-8')
print(predicted)

NameError: name 'predictor_xgb' is not defined

# END of Containts ===============

# 5.後片付け
予期せぬ課金を防ぐために、以下のリソースを削除します。

* SageMaker 推論エンドポイント
* ECR
* S3
* SageMakerノートブックインスタンス

# 参考
* SageMaker のtrainingジョブを理解する
    * https://github.com/aws-samples/aws-ml-jp/tree/main/sagemaker/sagemaker-traning/tutorial
* SageMaker-Pytorth training Toolkit
    * https://github.com/aws/sagemaker-pytorch-training-toolkit/
* SageMaker-Pytorch Inference Toolkit
    * https://github.com/aws/sagemaker-pytorch-inference-toolkit
* SageMaker Inference Toolkit
    * https://docs.aws.amazon.com/sagemaker/latest/dg/amazon-sagemaker-toolkits.html
    * https://github.com/aws/sagemaker-inference-toolkit